In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import re
from torchvision.ops import box_iou
import torch
import os
import re
from collections import OrderedDict
import json
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import defaultdict

In [ ]:
import json
from collections import Counter

with open('/content/chest_ima_train_full.json', 'r') as f:
    data = json.load(f)

unique_image_ids = len(set(item['id'] for item in data))

print(f"Number of unique image IDs: {unique_image_ids}")


In [ ]:
import json
from collections import Counter

with open('/content/eval_all_batches_ep1_multipleInstructions_formatted.json', 'r') as f:
    data = json.load(f)
unique_image_ids = len(set(item['idx'] for item in data))

print(f"Number of unique image IDs: {unique_image_ids}")


In [ ]:
import pandas as pd

df = pd.read_csv('/content/llava_train_01.csv')
unique_image_count = df['image_id'].nunique()

print(f"Number of unique image IDs: {unique_image_count}")


In [ ]:
import pandas as pd

df = pd.read_csv('/content/llava_valid_01.csv')
unique_image_count = df['image_id'].nunique()

print(f"Number of unique image IDs: {unique_image_count}")


In [ ]:
image_id_counts = df['image_id'].value_counts()

# Check if all image_id counts are 36
all_counts_36 = (image_id_counts == 36).all()

if all_counts_36:
    print("Each image_id appears exactly 36 times.")
else:
    print("Not all image_ids appear 36 times. Here are the counts:")
    print(image_id_counts[image_id_counts != 36])

In [ ]:
def load_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        print(len(data))
    filtered_data = []
    for json_obj in data:
        has_zero_area_bbox = False
        for bbox in json_obj['bbx_dict'].values():
            x1, y1, x2, y2 = bbox
            area = (x2 - x1) * (y2 - y1)
            if area == 0:
                has_zero_area_bbox = True
                break
        if not has_zero_area_bbox:
            filtered_data.append(json_obj)

    return filtered_data

In [ ]:
def compute_zero_gt(data):
  total_images = 0
  images_with_zero_area = 0

  for json_obj in data:
      total_images += 1
      has_zero_area_bbox = False
      for bbox in json_obj['bbx_dict'].values():
          x1, y1, x2, y2 = bbox
          area = (x2 - x1) * (y2 - y1)
          if area == 0:
              has_zero_area_bbox = True
              break
      if has_zero_area_bbox:
          images_with_zero_area += 1

  share_of_images_with_zero_area = images_with_zero_area / total_images if total_images > 0 else 0
  print("Total number of images:", total_images)
  print("Number of images with zero-area bounding boxes:", images_with_zero_area)
  print("Share of images with zero-area bounding boxes:", share_of_images_with_zero_area)


In [ ]:
ep1_multipleInstructions_formatted = load_json_file('/content/eval_all_batches_ep1_multipleInstructions_formatted.json')
ep1_multipleInstructions_weighted_formatted = load_json_file('/content/eval_all_batches_ep1_multipleInstructions_weighted_formatted.json')
ep1_singleInstruction_formatted = load_json_file('/content/eval_all_batches_ep1_singleInstruction_formatted.json')
ep1_singleInstruction_weighted_formatted = load_json_file('/content/eval_all_batches_ep1_singleInstruction_weighted_formatted.json')
ep2_multipleInstructions_formatted = load_json_file('/content/eval_all_batches_ep2_multipleInstructions_formatted.json')
ep2_singleInstruction_formatted = load_json_file('/content/eval_all_batches_ep2_singleInstruction_formatted.json')
ep3_multipleInstructions_formatted = load_json_file('/content/eval_all_batches_ep3_multipleInstructions_formatted.json')
ep3_singleInstruction_formatted = load_json_file('/content/eval_all_batches_ep3_singleInstruction_formatted.json')

In [ ]:
ep1_multipleInstructions_formatted

In [ ]:
def calculate_iou(bbox1, bbox2):
    """
    Calculate the IOU of two bounding boxes
    """
    x1 = max(bbox1[0], bbox2[0])
    y1 = max(bbox1[1], bbox2[1])
    x2 = min(bbox1[2], bbox2[2])
    y2 = min(bbox1[3], bbox2[3])

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    bbox1_area = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
    bbox2_area = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])

    union_area = bbox1_area + bbox2_area - intersection_area

    if union_area == 0:
        return 0.0

    return intersection_area / union_area

In [ ]:
anatomies = [
    "abdomen", "aortic arch", "cardiac silhouette", "carina", "cavoatrial junction",
    "descending aorta", "left apical zone", "left cardiac silhouette", "left cardiophrenic angle",
    "left clavicle", "left costophrenic angle", "left hemidiaphragm", "left hilar structures",
    "left lower lung zone", "left lung", "left mid lung zone", "left upper abdomen",
    "left upper lung zone", "mediastinum", "right apical zone", "right atrium",
    "right cardiac silhouette", "right cardiophrenic angle", "right clavicle",
    "right costophrenic angle", "right hemidiaphragm", "right hilar structures",
    "right lower lung zone", "right lung", "right mid lung zone", "right upper abdomen",
    "right upper lung zone", "spine", "svc", "trachea", "upper mediastinum"
]

In [ ]:
def is_valid_bbox(bbox):
    """
    Check if a bounding box is valid (i.e., has non-zero width and height).
    """
    if len(bbox) != 4:
        return False
    x1, y1, x2, y2 = bbox
    return (x2 - x1 > 0) and (y2 - y1 > 0)

In [ ]:
def extract_bbox_from_text(text):
    bbox_texts = re.findall(r"\[(.*?)\]", text)
    bbox_matches = []

    for bbox in bbox_texts:
        try:
            bbox_floats = list(map(float, bbox.split(',')))
            bbox_matches.append(bbox_floats)
        except ValueError as e:
            print(f"Error parsing bbox: {bbox}. Skipping it. Error: {e}")

    return bbox_matches

In [ ]:
def get_best_bbox_for_anatomy(bbox_matches, gt_bbox):
    max_iou = -1
    best_bbox = None

    for predicted_bbox in bbox_matches:
        iou = calculate_iou(predicted_bbox, gt_bbox)
        if iou > max_iou:
            max_iou = iou
            best_bbox = predicted_bbox

    return best_bbox


In [ ]:
def print_key_mismatch_info(anatomy_to_bbox, bbx_dict, original_text):
    print("Warning: anatomy_to_bbox and bbx_dict have different keys!")
    print(f"anatomy_to_bbox keys: {set(anatomy_to_bbox.keys())}")
    print(f"bbx_dict keys: {set(bbx_dict.keys())}")
    print(f"Original Text: {original_text}")
    print(f"bbx_dict: {bbx_dict}")

In [ ]:
def check_key_length_mismatch(anatomy_to_bbox, bbx_dict, original_text):
    if set(anatomy_to_bbox.keys()) != set(bbx_dict.keys()):
        print_key_mismatch_info(anatomy_to_bbox, bbx_dict, original_text)

    if len(anatomy_to_bbox) != len(bbx_dict):
        print(f"Warning: Length mismatch between anatomy_to_bbox ({len(anatomy_to_bbox)}) and bbx_dict ({len(bbx_dict)})")

In [ ]:
def assign_bboxes_to_all_anatomies(anatomy_matches, bbox_matches, bbx_dict):
    anatomy_to_bbox = {}

    for anatomy in anatomy_matches:
        gt_bbox = bbx_dict.get(anatomy)

        if gt_bbox:
            best_bbox = get_best_bbox_for_anatomy(bbox_matches, gt_bbox)
        else:
            best_bbox = bbox_matches[0] if bbox_matches else None

        if best_bbox:
            if anatomy in anatomy_to_bbox:
                if best_bbox not in anatomy_to_bbox[anatomy]:
                    anatomy_to_bbox[anatomy].append(best_bbox)
            else:
                anatomy_to_bbox[anatomy] = [best_bbox]

    return anatomy_to_bbox

In [ ]:
def extract_bboxes(text, bbx_dict):
    """
    Extract predicted bounding boxes and return a dictionary in which the key is the anatomy
    and the value is a tuple of lists containing multiple bounding boxes for that anatomy.
    If the number of anatomies and bounding boxes do not match, all bounding boxes will be assigned
    to each anatomy, but this will not be counted as a mismatch.
    Invalid bounding boxes (those with zero width or height) are filtered out.
    """
    mismatch_count = 0
    text = text.lower()

    anatomy_regex = re.compile(r'\b(' + '|'.join(map(re.escape, sorted(bbx_dict.keys(), key=len, reverse=True))) + r')\b')
    anatomy_matches = anatomy_regex.findall(text)

    bbox_matches = extract_bbox_from_text(text)
    if not anatomy_matches:
        if len(bbx_dict) == len(bbox_matches):
            anatomy_matches = list(bbx_dict.keys())
        else:
            # print("Mismatch between bbx_dict keys and bbox_matches length.")
            mismatch_count += 1
            return {}, mismatch_count
    anatomy_to_bbox = assign_bboxes_to_all_anatomies(anatomy_matches, bbox_matches, bbx_dict)

    # for anatomy, bboxes in anatomy_to_bbox.items():
    #     if len(bboxes) > 1:
    #         print(f"Warning: Anatomy '{anatomy}' has more than one bounding box: {bboxes}")
    for anatomy, bboxes in anatomy_to_bbox.items():
        if len(bboxes) > 1:
            anatomy_to_bbox[anatomy] = max(bboxes, key=lambda bbox: calculate_iou(bbox, bbx_dict.get(anatomy, bboxes[0])))
        else:
            anatomy_to_bbox[anatomy] = bboxes[0]
    check_key_length_mismatch(anatomy_to_bbox, bbx_dict, text)

    return anatomy_to_bbox, mismatch_count

# Calculate AP, AR and Average IOU

In [ ]:
def extract_dict(data):
    bbx_dict_list = []
    anatomy_to_bbox_list = []

    for item in data:
        bbx_dict = item['bbx_dict']
        bbx_dict_list.append(bbx_dict)
        anatomy_to_bbox, mismatch_count = extract_bboxes(item['output'], bbx_dict)
        anatomy_to_bbox_list.append(anatomy_to_bbox)

    return bbx_dict_list, anatomy_to_bbox_list

In [ ]:
def calculate_metrics_for_anatomy(anatomy, anatomy_to_bbox_list, bbx_dict_list, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    iou_sum = 0
    iou_count = 0

    for anatomy_to_bbox, bbx_dict in zip(anatomy_to_bbox_list, bbx_dict_list):
        if anatomy in anatomy_to_bbox:
            predicted_bbox = anatomy_to_bbox[anatomy]
            if anatomy in bbx_dict:
                gt_bbox = bbx_dict[anatomy]
                iou = calculate_iou(predicted_bbox, gt_bbox)
                if iou >= iou_threshold:
                    tp += 1
                else:
                    fp += 1
                iou_sum += iou
                iou_count += 1
            else:
                fp += 1
        else:
            if anatomy in bbx_dict:
                fn += 1

    average_iou = iou_sum / iou_count if iou_count > 0 else 0.0
    return tp, fp, fn, average_iou

In [ ]:
def calculate_precision_recall(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

In [ ]:
def calculate_ap_ar_iou_for_anatomies(anatomies, anatomy_to_bbox_list, bbx_dict_list, iou_threshold=0.5):
    anatomy_metrics = {}
    total_tp, total_fp, total_fn = 0, 0, 0
    iou_sum, iou_count = 0.0, 0
    ap_sum = 0
    for anatomy in anatomies:
        tp, fp, fn, average_iou = calculate_metrics_for_anatomy(anatomy, anatomy_to_bbox_list, bbx_dict_list, iou_threshold)
        precision, recall = calculate_precision_recall(tp, fp, fn)

        anatomy_metrics[anatomy] = {
            'AP': precision,
            'AR': recall,
            'Average IOU': average_iou,
            'TP': tp,
            'FP': fp,
            'FN': fn
        }

        total_tp += tp
        total_fp += fp
        total_fn += fn
        ap_sum += precision
        iou_sum += average_iou
        iou_count += 1

    mAP = ap_sum / len(anatomies) if len(anatomies) > 0 else 0
    mAR = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    mean_average_iou = iou_sum / iou_count if iou_count > 0 else 0

    return anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn



In [ ]:
bbx_dict_list, anatomy_to_bbox_list = extract_dict(ep1_multipleInstructions_formatted)

In [ ]:
anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn = calculate_ap_ar_iou_for_anatomies(
    anatomies, anatomy_to_bbox_list, bbx_dict_list
)

for anatomy, metrics in anatomy_metrics.items():
    print(f"{anatomy}: AP = {metrics['AP']:.4f}, AR = {metrics['AR']:.4f}, Average IOU = {metrics['Average IOU']:.4f}, TP = {metrics['TP']}, FP = {metrics['FP']}, FN = {metrics['FN']}")


# Print overall metrics
print(f"Mean Average Precision (mAP): {mAP:.4f}")
print(f"Mean Average Recall (mAR): {mAR:.4f}")
print(f"Mean Average IOU: {mean_average_iou:.4f}")
print(f"Total True Positives (TP): {total_tp}")
print(f"Total False Positives (FP): {total_fp}")
print(f"Total False Negatives (FN): {total_fn}")

In [ ]:
bbx_dict_list, anatomy_to_bbox_list = extract_dict(ep1_multipleInstructions_weighted_formatted)

In [ ]:
anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn = calculate_ap_ar_iou_for_anatomies(
    anatomies, anatomy_to_bbox_list, bbx_dict_list
)

for anatomy, metrics in anatomy_metrics.items():
    print(f"{anatomy}: AP = {metrics['AP']:.4f}, AR = {metrics['AR']:.4f}, Average IOU = {metrics['Average IOU']:.4f}, TP = {metrics['TP']}, FP = {metrics['FP']}, FN = {metrics['FN']}")

print(f"Mean Average Precision (mAP): {mAP:.4f}")
print(f"Mean Average Recall (mAR): {mAR:.4f}")
print(f"Mean Average IOU: {mean_average_iou:.4f}")
print(f"Total True Positives (TP): {total_tp}")
print(f"Total False Positives (FP): {total_fp}")
print(f"Total False Negatives (FN): {total_fn}")

In [ ]:
bbx_dict_list, anatomy_to_bbox_list = extract_dict(ep1_singleInstruction_formatted)

In [ ]:
anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn = calculate_ap_ar_iou_for_anatomies(
    anatomies, anatomy_to_bbox_list, bbx_dict_list
)

for anatomy, metrics in anatomy_metrics.items():
    print(f"{anatomy}: AP = {metrics['AP']:.4f}, AR = {metrics['AR']:.4f}, Average IOU = {metrics['Average IOU']:.4f}, TP = {metrics['TP']}, FP = {metrics['FP']}, FN = {metrics['FN']}")


print(f"Mean Average Precision (mAP): {mAP:.4f}")
print(f"Mean Average Recall (mAR): {mAR:.4f}")
print(f"Mean Average IOU: {mean_average_iou:.4f}")
print(f"Total True Positives (TP): {total_tp}")
print(f"Total False Positives (FP): {total_fp}")
print(f"Total False Negatives (FN): {total_fn}")

In [ ]:
bbx_dict_list, anatomy_to_bbox_list = extract_dict(ep1_singleInstruction_weighted_formatted)

In [ ]:
anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn = calculate_ap_ar_iou_for_anatomies(
    anatomies, anatomy_to_bbox_list, bbx_dict_list
)

for anatomy, metrics in anatomy_metrics.items():
    print(f"{anatomy}: AP = {metrics['AP']:.4f}, AR = {metrics['AR']:.4f}, Average IOU = {metrics['Average IOU']:.4f}, TP = {metrics['TP']}, FP = {metrics['FP']}, FN = {metrics['FN']}")


print(f"Mean Average Precision (mAP): {mAP:.4f}")
print(f"Mean Average Recall (mAR): {mAR:.4f}")
print(f"Mean Average IOU: {mean_average_iou:.4f}")
print(f"Total True Positives (TP): {total_tp}")
print(f"Total False Positives (FP): {total_fp}")
print(f"Total False Negatives (FN): {total_fn}")

In [ ]:
bbx_dict_list, anatomy_to_bbox_list = extract_dict(ep2_singleInstruction_formatted)

In [ ]:
anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn = calculate_ap_ar_iou_for_anatomies(
    anatomies, anatomy_to_bbox_list, bbx_dict_list
)

for anatomy, metrics in anatomy_metrics.items():
    print(f"{anatomy}: AP = {metrics['AP']:.4f}, AR = {metrics['AR']:.4f}, Average IOU = {metrics['Average IOU']:.4f}, TP = {metrics['TP']}, FP = {metrics['FP']}, FN = {metrics['FN']}")

print(f"Mean Average Precision (mAP): {mAP:.4f}")
print(f"Mean Average Recall (mAR): {mAR:.4f}")
print(f"Mean Average IOU: {mean_average_iou:.4f}")
print(f"Total True Positives (TP): {total_tp}")
print(f"Total False Positives (FP): {total_fp}")
print(f"Total False Negatives (FN): {total_fn}")

In [ ]:
bbx_dict_list, anatomy_to_bbox_list = extract_dict(ep3_multipleInstructions_formatted)

In [ ]:
anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn = calculate_ap_ar_iou_for_anatomies(
    anatomies, anatomy_to_bbox_list, bbx_dict_list
)

for anatomy, metrics in anatomy_metrics.items():
    print(f"{anatomy}: AP = {metrics['AP']:.4f}, AR = {metrics['AR']:.4f}, Average IOU = {metrics['Average IOU']:.4f}, TP = {metrics['TP']}, FP = {metrics['FP']}, FN = {metrics['FN']}")


print(f"Mean Average Precision (mAP): {mAP:.4f}")
print(f"Mean Average Recall (mAR): {mAR:.4f}")
print(f"Mean Average IOU: {mean_average_iou:.4f}")
print(f"Total True Positives (TP): {total_tp}")
print(f"Total False Positives (FP): {total_fp}")
print(f"Total False Negatives (FN): {total_fn}")

In [ ]:
bbx_dict_list, anatomy_to_bbox_list = extract_dict(ep3_singleInstruction_formatted)

In [ ]:
anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn = calculate_ap_ar_iou_for_anatomies(
    anatomies, anatomy_to_bbox_list, bbx_dict_list
)

for anatomy, metrics in anatomy_metrics.items():
    print(f"{anatomy}: AP = {metrics['AP']:.4f}, AR = {metrics['AR']:.4f}, Average IOU = {metrics['Average IOU']:.4f}, TP = {metrics['TP']}, FP = {metrics['FP']}, FN = {metrics['FN']}")

print(f"Mean Average Precision (mAP): {mAP:.4f}")
print(f"Mean Average Recall (mAR): {mAR:.4f}")
print(f"Mean Average IOU: {mean_average_iou:.4f}")
print(f"Total True Positives (TP): {total_tp}")
print(f"Total False Positives (FP): {total_fp}")
print(f"Total False Negatives (FN): {total_fn}")

In [ ]:
bbx_dict_list[:2]

In [ ]:
anatomy_to_bbox_list[:2]

# Output the metrics based on the quantity of anatomies

In [ ]:
def categorize_by_length(bbx_dict_list):
    categorized_indices = defaultdict(list)

    for idx, bbx_dict in enumerate(bbx_dict_list):
        length = len(bbx_dict)
        if 1 <= length <= 5:
            categorized_indices[length].append(idx)

    return categorized_indices

In [ ]:
def calculate_metrics_by_category(anatomies, anatomy_to_bbox_list, bbx_dict_list, categorized_indices, iou_threshold=0.5):
    metrics_by_category = {}
    for length, indices in categorized_indices.items():
        sub_anatomy_to_bbox_list = [anatomy_to_bbox_list[idx] for idx in indices]
        sub_bbx_dict_list = [bbx_dict_list[idx] for idx in indices]
        anatomy_metrics, mAP, mAR, mean_average_iou, total_tp, total_fp, total_fn = calculate_ap_ar_iou_for_anatomies(anatomies, sub_anatomy_to_bbox_list, sub_bbx_dict_list, iou_threshold)
        metrics_by_category[length] = {
            'mAP': mAP,
            'mAR': mAR,
            'Mean Average IOU': mean_average_iou,
            'TP': total_tp,
            'FP': total_fp,
            'FN': total_fn,
            'Anatomy Metrics': anatomy_metrics
        }

    return metrics_by_category

In [ ]:
def main(data, anatomies):
    bbx_dict_list, anatomy_to_bbox_list = extract_dict(data)
    categorized_indices = categorize_by_length(bbx_dict_list)
    metrics_by_category = calculate_metrics_by_category(anatomies, anatomy_to_bbox_list, bbx_dict_list, categorized_indices)
    for length, metrics in metrics_by_category.items():
        # print(metrics)
        print(f"Category with {length} anatomies:")
        print(f"  Mean Average Precision (mAP): {metrics['mAP']:.4f}")
        print(f"  Mean Average Recall (mAR): {metrics['mAR']:.4f}")
        print(f"  Mean Average IOU: {metrics['Mean Average IOU']:.4f}")
        print()

In [ ]:
main(ep2_singleInstruction_formatted, anatomies)

# Output the metrics according to the size

In [ ]:
ap_dict = {
    "abdomen": 0.9848,
    "aortic arch": 0.1295,
    "cardiac silhouette": 0.8905,
    "carina": 0.1032,
    "cavoatrial junction": 0.5954,
    "descending aorta": 0.6449,
    "left apical zone": 0.4026,
    "left cardiac silhouette": 0.8907,
    "left cardiophrenic angle": 0.0897,
    "left clavicle": 0.4246,
    "left costophrenic angle": 0.0671,
    "left hemidiaphragm": 0.5543,
    "left hilar structures": 0.8063,
    "left lower lung zone": 0.8731,
    "left lung": 0.9794,
    "left mid lung zone": 0.7504,
    "left upper abdomen": 0.9825,
    "left upper lung zone": 0.8217,
    "mediastinum": 0.9510,
    "right apical zone": 0.4060,
    "right atrium": 0.7275,
    "right cardiac silhouette": 0.8214,
    "right cardiophrenic angle": 0.2614,
    "right clavicle": 0.4525,
    "right costophrenic angle": 0.0771,
    "right hemidiaphragm": 0.6305,
    "right hilar structures": 0.8002,
    "right lower lung zone": 0.9081,
    "right lung": 0.9724,
    "right mid lung zone": 0.7230,
    "right upper abdomen": 0.9596,
    "right upper lung zone": 0.7870,
    "spine": 0.9757,
    "svc": 0.6436,
    "trachea": 0.8300,
    "upper mediastinum": 0.9013
}


In [ ]:
large_anatomy = ['abdomen', 'left lung', 'mediastinum', 'right lung', 'spine']
medium_large_anatomy = ['cardiac silhouette', 'left lower lung zone', 'left upper abdomen', 'right upper abdomen']
medium_small_anatomy = ['descending aorta', 'left apical zone', 'left cardiac silhouette', 'left clavicle',
                        'left hemidiaphragm', 'left mid lung zone', 'left upper lung zone', 'right apical zone',
                        'right clavicle', 'right hemidiaphragm', 'right lower lung zone', 'right mid lung zone',
                        'right upper lung zone', 'trachea', 'upper mediastinum']
small_anatomy = ['aortic arch', 'carina', 'cavoatrial junction', 'left cardiophrenic angle', 'left costophrenic angle',
                 'left hilar structures', 'right atrium', 'right cardiac silhouette', 'right cardiophrenic angle',
                 'right costophrenic angle', 'right hilar structures', 'svc']

In [ ]:
def calculate_average_ap(anatomy_list, ap_dict):
    total_ap = sum(ap_dict[anatomy] for anatomy in anatomy_list)
    return total_ap / len(anatomy_list)

In [ ]:
average_ap_large = calculate_average_ap(large_anatomy, ap_dict)
average_ap_medium_large = calculate_average_ap(medium_large_anatomy, ap_dict)
average_ap_medium_small = calculate_average_ap(medium_small_anatomy, ap_dict)
average_ap_small = calculate_average_ap(small_anatomy, ap_dict)

print("Average AP for large anatomy:", average_ap_large)
print("Average AP for medium-large anatomy:", average_ap_medium_large)
print("Average AP for medium-small anatomy:", average_ap_medium_small)
print("Average AP for small anatomy:", average_ap_small)

In [ ]:
import pandas as pd
import statsmodels.api as sm

anatomy_data = {
    'anatomy': ["abdomen", "aortic arch", "cardiac silhouette", "carina", "cavoatrial junction",
                "descending aorta", "left apical zone", "left cardiac silhouette", "left cardiophrenic angle",
                "left clavicle", "left costophrenic angle", "left hemidiaphragm", "left hilar structures",
                "left lower lung zone", "left lung", "left mid lung zone", "left upper abdomen",
                "left upper lung zone", "mediastinum", "right apical zone", "right atrium",
                "right cardiac silhouette", "right cardiophrenic angle", "right clavicle",
                "right costophrenic angle", "right hemidiaphragm", "right hilar structures",
                "right lower lung zone", "right lung", "right mid lung zone", "right upper abdomen",
                "right upper lung zone", "spine", "svc", "trachea", "upper mediastinum"],
    'performance_score': [0.9791, 0.1839, 0.8875, 0.1280, 0.5980, 0.5946, 0.3859, 0.8876,
                          0.0805, 0.3850, 0.0769, 0.3737, 0.8524, 0.8753, 0.9810, 0.8231,
                          0.9764, 0.8040, 0.9501, 0.3610, 0.6857, 0.7990, 0.1615, 0.3807,
                          0.0695, 0.4424, 0.8601, 0.9013, 0.9734, 0.8370, 0.9540, 0.8383,
                          0.9794, 0.7012, 0.8167, 0.8866],
    'area': [0.261785, 0.007442, 0.104342, 0.001889, 0.008393, 0.032765, 0.034129, 0.067855,
             0.012220, 0.044184, 0.008193, 0.048669, 0.029804, 0.078252, 0.213593, 0.041016,
             0.115691, 0.062117, 0.204767, 0.034344, 0.017663, 0.026029, 0.006754, 0.044548,
             0.008132, 0.053263, 0.028140, 0.072208, 0.208074, 0.037906, 0.129990, 0.062344,
             0.162061, 0.015124, 0.051271, 0.048524],
    'size_category': ["Very Large", "Small", "Very Large", "Small", "Small", "Medium", "Medium",
                      "Large", "Small", "Medium", "Small", "Medium", "Medium", "Large", "Very Large",
                      "Medium", "Very Large", "Large", "Very Large", "Medium", "Small", "Medium",
                      "Small", "Medium", "Small", "Large", "Medium", "Large", "Very Large", "Medium",
                      "Very Large", "Large", "Very Large", "Small", "Large", "Medium"],
    'occurrence': [42503, 14728, 9956, 14600, 11580, 775, 3253, 23843, 47386, 25265, 3252,
                   5863, 3888, 26355, 19178, 47227, 3106, 330, 2441, 2676, 14083, 3654, 14611,
                   5547, 1027, 30208, 2644, 14263, 7560, 19703, 1124, 43026, 9330, 7342, 11908,
                   5637]
}

df = pd.DataFrame(anatomy_data)

df = pd.get_dummies(df, columns=['size_category'], drop_first=True)

X = df[['area', 'occurrence']]
y = df['performance_score']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression

pearson_corr, pearson_p_value = pearsonr(df['performance_score'], df['area'])
spearman_corr, spearman_p_value = spearmanr(df['performance_score'], df['area'])

X = df['area'].values.reshape(-1, 1)
y = df['performance_score'].values
regression_model = LinearRegression()
regression_model.fit(X, y)
predicted_values = regression_model.predict(X)
plt.figure(figsize=(10, 6))
plt.scatter(df['area'], df['performance_score'], color='blue', alpha=0.6, label='Anatomy Performance')
plt.plot(df['area'], predicted_values, color='red', label='Regression Line')
plt.xlabel('Area')
plt.ylabel('Average Precision (AP)')
plt.title('Relationship between Area and Average Precision (AP)')
plt.legend()

plt.text(0.05, 0.95, f"Pearson Correlation: {pearson_corr:.4f} (P-value: {pearson_p_value:.10f})\n"
                     f"Spearman Correlation: {spearman_corr:.4f} (P-value: {spearman_p_value:.10f})",
         ha='left', va='top', transform=plt.gca().transAxes, fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.5))


plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

models = [
    "LLaVA-Imbal-High-e1", "LLaVA-Bal-High-e1", "LLaVA-Imbal-Low-e1",
    "LLaVA-Bal-Low-e1", "LLaVA-Bal-Low-e2", "LLaVA-Bal-High-e3",
    "LLaVA-Bal-Low-e3"
]

area_values = np.array([
    0.261785, 0.007442, 0.104342, 0.001889, 0.008393, 0.032765, 0.034129, 0.067855,
    0.012220, 0.044184, 0.008193, 0.048669, 0.029804, 0.078252, 0.213593, 0.041016,
    0.115691, 0.062117, 0.204767, 0.034344, 0.017663, 0.026029, 0.006754, 0.044548,
    0.008132, 0.053263, 0.028140, 0.072208, 0.208074, 0.037906, 0.129990, 0.062344,
    0.162061, 0.015124, 0.051271, 0.048524
])

ap_data = [
    # LLaVA-Imbal-Multi-e1
    [
        0.9812, 0.0421, 0.8890, 0.0242, 0.5194, 0.5786, 0.2974, 0.8840,
        0.1081, 0.3427, 0.0861, 0.4884, 0.7861, 0.8776, 0.9763, 0.6536,
        0.9769, 0.7325, 0.9460, 0.2832, 0.6475, 0.7436, 0.1552, 0.3072,
        0.0771, 0.5426, 0.6619, 0.8765, 0.9634, 0.5916, 0.9519, 0.6462,
        0.9746, 0.4468, 0.7763, 0.7945
    ],
    # LLaVA-Bal-Multi-e1
    [
        0.9859, 0.1429, 0.8951, 0.1006, 0.6198, 0.6495, 0.3318, 0.8927,
        0.0862, 0.3905, 0.0834, 0.4381, 0.8365, 0.8780, 0.9810, 0.7907,
        0.9810, 0.7854, 0.9531, 0.3467, 0.7071, 0.8167, 0.2241, 0.4320,
        0.0695, 0.4594, 0.8315, 0.9060, 0.9764, 0.7795, 0.9561, 0.8092,
        0.9788, 0.6848, 0.8250, 0.8952
    ],
    # LLaVA-Imbal-Single-e1
    [
        0.9791, 0.1839, 0.8875, 0.1280, 0.5980, 0.5946, 0.3859, 0.8876,
        0.0805, 0.3850, 0.0769, 0.3737, 0.8524, 0.8753, 0.9810, 0.8231,
        0.9764, 0.8040, 0.9501, 0.3610, 0.6857, 0.7990, 0.1615, 0.3807,
        0.0695, 0.4424, 0.8601, 0.9013, 0.9734, 0.8370, 0.9540, 0.8383,
        0.9794, 0.7012, 0.8167, 0.8866
    ],
    # LLaVA-Bal-Single-e1
    [
        0.9843, 0.1519, 0.8946, 0.0964, 0.6192, 0.6901, 0.3620, 0.8985,
        0.0841, 0.3927, 0.0877, 0.4743, 0.8503, 0.8764, 0.9810, 0.8017,
        0.9805, 0.8206, 0.9566, 0.3526, 0.6882, 0.8136, 0.1862, 0.4033,
        0.0792, 0.4667, 0.8345, 0.9050, 0.9784, 0.7959, 0.9582, 0.8187,
        0.9804, 0.6952, 0.8300, 0.8967
    ],
    # LLaVA-Bal-Single-e2
    [
        0.9848, 0.1295, 0.8905, 0.1032, 0.5954, 0.6449, 0.4026, 0.8907,
        0.0897, 0.4246, 0.0671, 0.5543, 0.8063, 0.8731, 0.9794, 0.7504,
        0.9825, 0.8217, 0.9510, 0.4060, 0.7275, 0.8214, 0.2614, 0.4525,
        0.0771, 0.6305, 0.8002, 0.9081, 0.9724, 0.7230, 0.9596, 0.7870,
        0.9757, 0.6436, 0.8300, 0.9013
    ],
    # LLaVA-Bal-Multi-e3
    [
        0.9796, 0.0448, 0.8890, 0.0369, 0.6110, 0.5493, 0.2042, 0.8804,
        0.0697, 0.2783, 0.0747, 0.3929, 0.7415, 0.8737, 0.9757, 0.6787,
        0.9753, 0.6304, 0.9435, 0.2440, 0.7351, 0.8157, 0.2041, 0.3153,
        0.0700, 0.4119, 0.8002, 0.9039, 0.9709, 0.7057, 0.9518, 0.6950,
        0.9757, 0.6365, 0.8117, 0.8703
    ],
    # LLaVA-Bal-Single-e3
    [
        0.9801, 0.0410, 0.8900, 0.0500, 0.6255, 0.5247, 0.1693, 0.8835,
        0.0651, 0.2904, 0.0736, 0.3778, 0.7521, 0.8665, 0.9762, 0.6942,
        0.9743, 0.6172, 0.9470, 0.2474, 0.7441, 0.8214, 0.2341, 0.3137,
        0.0710, 0.3995, 0.8084, 0.9076, 0.9719, 0.7231, 0.9540, 0.7209,
        0.9746, 0.6358, 0.8123, 0.8815
    ]
]


fig, axes = plt.subplots(1, 7, figsize=(16, 3), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.3, wspace=0.3)
axes = axes.flatten()

for i, (model, ap_values) in enumerate(zip(models, ap_data)):
    ap_values = np.array(ap_values)

    regression_model = LinearRegression()
    area_values_reshaped = area_values.reshape(-1, 1)
    regression_model.fit(area_values_reshaped, ap_values)
    predicted_ap_values = regression_model.predict(area_values_reshaped)

    intercept = regression_model.intercept_
    slope = regression_model.coef_[0]

    axes[i].scatter(area_values, ap_values, color='blue', s=10)
    axes[i].plot(area_values, predicted_ap_values, color='red')
    axes[i].set_title(f"{model}")
    axes[i].set_xlabel('Area of Anatomy')
    axes[i].set_ylabel('Average Precision')
    axes[i].grid(False)
    axes[i].set_facecolor("#f8f8f8")

    axes[i].text(0.05, 0.95, f"Intercept: {intercept:.2f}\nSlope: {slope:.2f}",
                 transform=axes[i].transAxes, fontsize=8, verticalalignment='top')
plt.savefig("regression_plots.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
from scipy.stats import pearsonr, spearmanr

for i, (model, ap_values) in enumerate(zip(models, ap_data)):
    ap_values = np.array(ap_values)

    pearson_corr, pearson_p_value = pearsonr(area_values, ap_values)
    spearman_corr, spearman_p_value = spearmanr(area_values, ap_values)

    print(f"{model}:")
    print(f"  Pearson Correlation: {pearson_corr:.2f}, P-value: {pearson_p_value:.10f}")
    print(f"  Spearman Correlation: {spearman_corr:.2f}, P-value: {spearman_p_value:.10f}")
    print("-" * 50)
